# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.stats.api as sms

C:\Users\sixsi\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race == 'b'].call)

157.0

In [5]:
data.race.value_counts()

w    2435
b    2435
Name: race, dtype: int64

## 1. What test is appropriate for this problem? Does CLT apply?
- A two-sample test is appropriate for this exercise
- The centrl limit theorm does apply in this case:
    * A large overall sample size, 4870.
    * Evenly split between 'b' and 'w'

## 2. What are the null and alternate hypotheses?
- Ho: mean(black call backs) - mean(white call backs) = 0
- Ha: mean(black call backs) - mean(white call backs) != 0

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']

## 3. Compute margin of error, confidence interval, and p-value. 
- Try using both the bootstrapping and the frequentist statistical approaches.

In [7]:
print('White call-back rate: ', w.call.mean())
print('Black call-back rate: ', b.call.mean())
print('Mean difference: ', w.call.mean() - b.call.mean())

White call-back rate:  0.09650924056768417
Black call-back rate:  0.0644763857126236
Mean difference:  0.03203285485506058


In [8]:
# First, create the bootstrap replicate function
def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

In [9]:
# create a function which generates many bootstrap replicates from the data set. 
def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data,func)

    return bs_replicates

In [10]:
# Use the bootstrap hypothesis test

# Compute the difference in b and w call backs: diff_means
diff_means = b.call.mean() - w.call.mean()

# Compute mean of pooled data: mean_count
mean_count = data.call.mean()

# Generate shifted data sets
b_shifted = b.call - b.call.mean() + mean_count
w_shifted = w.call - w.call.mean() + mean_count

# generate bootstrap replicates
bs_b = draw_bs_reps(b_shifted, np.mean, size = 10000)
bs_w = draw_bs_reps(w_shifted, np.mean, size = 10000)

# Get replicates of difference of means: bs_replicates
bs_replicates = bs_b - bs_w

# Compute and print p-value: p
p = np.sum(bs_replicates <= diff_means) / len(bs_replicates)
print('p-value =', p)

p-value = 0.0001


In [11]:
# Using the bootstrap approach as before, look at the 95% confidence interval of the 10,000 replicates
np.percentile(bs_replicates, [2.5, 97.5])

array([-0.01560573,  0.01560578])

### Not sure why the bootstrap method did not work

In [12]:
# Using statsmodels.stats.api as found on Stackoverflow
comp_means = sms.CompareMeans(sms.DescrStatsW(w.call), sms.DescrStatsW(b.call))
comp_means.tconfint_diff(usevar='unequal')

(0.016770673983991798, 0.04729503443489937)

Margin of error = z-score * standard error

- for a 95% CI z = 1.96
- standard error = std / n**5


In [13]:
moe = 1.96 * (data.call.std()/np.sqrt(len(data)))
print('Margin of Error = ',moe)

Margin of Error =  0.007641652993837378


## 4. Write a story describing the statistical significance in the context or the original problem.

Because of the low p-value we can conclude there is a difference in the call back rate for individuals with black-sounding and white-sounding names. On average this difference is 3.2%, and based upon a 95% confidence interval, could be as high as 4.7% or as low as 1.7%

## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Race could be the most important factor, but we would not know unless we examined the affects of the other variables on the call-back rate, ie PCA or some other method